In [139]:
import pyspark
import numpy as np
from pyspark import SparkContext, SparkConf
from tqdm import tqdm

In [126]:
data = np.load('reduced_data.npy')

In [3]:
conf = SparkConf().setAppName("Prova").setMaster("local")
sc = SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/08 11:23:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [143]:
points = list(zip(data[:,0], data[:,1]))
rdd = sc.parallelize(points)
cent = rdd.takeSample(withReplacement=False, num=3) # initial random centroid

22/12/08 12:57:30 WARN TaskSetManager: Stage 142 contains a task of very large size (2231 KiB). The maximum recommended task size is 1000 KiB.
22/12/08 12:57:31 WARN TaskSetManager: Stage 143 contains a task of very large size (2231 KiB). The maximum recommended task size is 1000 KiB.


In [144]:
def centroids(k, rdd, cent, withResult = False):
    belong = rdd.map(lambda x: (min_distance(x, cent), x)) # which cluster each data belongs to
    count = belong.countByKey()
    summation = belong.reduceByKey(lambda x, y: (x[0]+y[0],  x[1]+y[1])) # pairwise summation over the values
    cent = summation.map(lambda x: (np.array(x[1])/ count[x[0]]))
    if withResult:
        return cent.collect(), belong.collect()
    return cent.collect()


In [121]:
cent.collect()

[array([1.25644287e+08, 1.50812245e+05]),
 array([-3.19917338e+07,  1.16718702e+04]),
 array([-32538506.00464778,   -254164.68246679])]

In [145]:
l = []
for i in tqdm(range(20)):
    prev_cent = cent
    cent = centroids(3, rdd, cent)
    l.append(np.linalg.norm(np.array(prev_cent)-np.array(cent)))
cent, belong = centroids(3, rdd, cent, withResult = True)
print(l)

  0%|          | 0/20 [00:00<?, ?it/s]

22/12/08 12:57:42 WARN TaskSetManager: Stage 144 contains a task of very large size (2231 KiB). The maximum recommended task size is 1000 KiB.


22/12/08 12:57:44 WARN TaskSetManager: Stage 145 contains a task of very large size (2231 KiB). The maximum recommended task size is 1000 KiB.


  5%|▌         | 1/20 [00:03<01:13,  3.86s/it]                                  

22/12/08 12:57:45 WARN TaskSetManager: Stage 147 contains a task of very large size (2231 KiB). The maximum recommended task size is 1000 KiB.


...

100%|██████████| 20/20 [01:16<00:00,  3.83s/it]                                 


22/12/08 12:58:58 WARN TaskSetManager: Stage 204 contains a task of very large size (2231 KiB). The maximum recommended task size is 1000 KiB.


22/12/08 12:59:00 WARN TaskSetManager: Stage 205 contains a task of very large size (2231 KiB). The maximum recommended task size is 1000 KiB.


Traceback (most recent call last):                                              
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/pyspark/serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/pyspark/rdd.py", line 356, in __getnewargs__
    raise RuntimeError(
RuntimeError: It appears that you are attempting to broadcast an RDD or reference an RDD from an action or transformation. RDD transformations and actions can only be invoked by the driver, not inside of other transformations; for example, rdd1.map(lambda x: rdd2.values.count() * x) is 

PicklingError: Could not serialize object: RuntimeError: It appears that you are attempting to broadcast an RDD or reference an RDD from an action or transformation. RDD transformations and actions can only be invoked by the driver, not inside of other transformations; for example, rdd1.map(lambda x: rdd2.values.count() * x) is invalid because the values transformation and count action cannot be performed inside of the rdd1.map transformation. For more information, see SPARK-5063.

In [146]:
l

[178642881.37034005,
 936934310.5774746,
 1466296529.48351,
 4551743043.274725,
 1718174935.9809332,
 1256210751.2620702,
 1154198426.5036173,
 908787307.9042007,
 7946138952.874206,
 11013716754.95524,
 40409108583.830246,
 8613597399.600466,
 11847045604.321938,
 185296576.6318393,
 73367612.45411074,
 14643934.951504769,
 0.0,
 0.0,
 0.0,
 0.0]

In [4]:
points = list(zip(data[:,0], data[:,1]))

In [5]:
rdd = sc.parallelize(points)

In [6]:
cent = rdd.takeSample(withReplacement=False, num=k) # initial random centroid

22/12/08 11:23:45 WARN TaskSetManager: Stage 0 contains a task of very large size (2231 KiB). The maximum recommended task size is 1000 KiB.


22/12/08 11:23:46 WARN TaskSetManager: Stage 1 contains a task of very large size (2231 KiB). The maximum recommended task size is 1000 KiB.


In [26]:
def min_distance(x,c:list):
    x, c = np.array(x), np.array(c)
    dist = np.array(list(map(lambda a: np.linalg.norm(a - x), c)))
    return np.argmin(dist)

In [77]:
belong = rdd.map(lambda x: (min_distance(x, cent), x)) # which cluster each data belongs to

In [109]:
belong.toDF()

RuntimeError: RDD.toDF was called before SparkSession was initialized.

In [86]:
count = belong.countByKey()

22/12/08 12:17:12 WARN TaskSetManager: Stage 57 contains a task of very large size (2231 KiB). The maximum recommended task size is 1000 KiB.


In [88]:
count

defaultdict(int, {0: 11568, 2: 36735, 1: 8551})

In [90]:
summation = belong.reduceByKey(lambda x, y: (x[0]+y[0],  x[1]+y[1])) # pairwise summation over the values

In [91]:
summation.collect()

22/12/08 12:19:30 WARN TaskSetManager: Stage 58 contains a task of very large size (2231 KiB). The maximum recommended task size is 1000 KiB.


[(0, (1453453106713.901, 1744596046.9203742)),
 (2, (-1175216341868.1558, 428766152.8531649)),
 (1, (-278236764845.74316, -2173362199.7735476))]

In [103]:
prev_cent = cent
cent = summation.map(lambda x: (np.array(x[1])/ count[x[0]]))

In [104]:
cent.collect()

[array([1.25644287e+08, 1.50812245e+05]),
 array([-3.19917338e+07,  1.16718702e+04]),
 array([-32538506.00464778,   -254164.68246679])]